# Dynamic Programming.

It refers to simplifying a complicated problem by breaking it down into simpler sub-problem in a recursive manner. While some decision problems cannot be taken apart this way, decistons that span several points in time do often break apart recursively.

`Having optimal substructure`, a problem can be solved optimally by breaking it into sub-problems and then recursively finding the oprtimal solutions to the sub-problems.

If sub-problems can be nested recursively inside larger problems, so that dynamic programming methods are applicable, then there is a realation between the value of the larget problem and the values of the sub-problems.

**Dynamic programming helps us solve recursive problems with a highly-overlapping subproblem structure.** `Highly-overlapping` refers to the subproblems repeating again and again. In contrast, an alogrithm like mergesort recursively sorts `independent halves` of a list beforce combining the sorted halves. When the subproblems don't overlap, the algorithm is a divide-and-conquer alogrithm.

## Fibonacci numbers

As a reminder, the Fibonacci numbers are a sequence starting with 1, 1 where eath element in the sequence is the sume of the two previous elements: 1, 1, 2, 3, 5, 8, 13,....

In [1]:
def naive_fib(n):
    if n <= 1: return 1
    return naive_fib(n-1) + naive_fib(n-2)

In [33]:
naive_fib(10)

89

In [3]:
import timeit
timeit.timeit('naive_fib(20)', number=100, globals=globals())

0.2664149560732767

As we know, use the recursive way to calculate the Fibonacci numbers will occur overlapping calculation for certain subproblems. For example, when you want to know the value of `Fib(5)`, you need to calculate `Fib(4)` and `Fib(3)` first, but the `Fib(4)` will come from the value of `Fib(3)`. The fact is we need to produce the result of `Fib(3)` twice. The total runtime will be exponential in n as `O(2^n)`. As mentioned above, the recursive subproblem of produce Fibonacci number have an overlapping calculation, we can use the DP to improve our runtime.

**Using memoization**

In [38]:
def memo_fib(n, memo = {}):
    if n <= 1:
        # Store the fib(0) and fib(1) into memo.
        memo[n] = 1
    if n in memo:
        # If fib(n) in the memo, do not reproduce the same fib number.
        return memo[n]
    fib_n = memo_fib(n - 1, memo) + memo_fib(n - 2, memo)
    memo[n] = fib_n
    return fib_n

In [39]:
memo_fib(10)

89

In [40]:
timeit.timeit('memo_fib(100)', number=100, globals=globals())

0.00013074802700430155

As the time showing above, we have significantly reduced our runtime bound from `O(2^n)` to `O(n)`. Otherwise, the space-time will be `O(n)` because we store the result of the previous value of the Fibonacci number.

**Bottom-up approach**

The formula of the Fibonacci number can be write as `Fib(n) = Fib(n-1) + Fib(n+2)`. If we consider that a `Fib(i)` is the subproblem of `Fib(n)`, where `i <= n`, we can only need the result of `Fib(i-1)` and `Fib(i-2)` to produce its result. That means we only need two variables to store our values, and we can throw away the value before the `Fib(i-2)` as its no longer help us to calculate the `Fib(i)`.

In [41]:
def bottom_up_fib(n):
    if n <= 1:
        return 1
    fib1 = 1
    fib2 = 1
    for i in range(2, n+1):
        # the old "fib1" will be throwd and assign the old "fib2" to it.
        tmp = fib1
        fib1 = fib2
        fib2 = tmp + fib2
    return fib2

In [42]:
bottom_up_fib(10)

89

In [43]:
timeit.timeit('bottom_up_fib(100)', number=100, globals=globals())

0.0005412490572780371

# The House Robber Problem

In the **House Robber Problem**, you are a robber who has found a block of house to rob. Each house i has a non-negative v(i) worth of value inside that you can steal. However, due to the security systems of the houses are connected, you'll get caught if you rob two adjacent houses. What's the maximum value you can steal from the block?

**Example 1:**
```py
Input: [1,2,3,1]
Output: 4
Explanation: Rob house 1 (money = 1) and then rob house 3 (money = 3).
             Total amount you can rob = 1 + 3 = 4.
```
**Example 2:**
```py
Input: [2,7,9,3,1]
Output: 12
Explanation: Rob house 1 (money = 2), rob house 3 (money = 9) and rob house 5 (money = 1).
             Total amount you can rob = 2 + 9 + 1 = 12.
```
Let's use the same steps solve this problem like Fibonacci.

1. The first step in solveing a dynamic programming problem is **identifying the subproblems**. Whenever we encounter a house i, we have two choices:
  * Steal from the house i add the `v[i]` into your wealth, and then compare it with the value of stealing the house up to `i - 2`, because you can not steal the adjacent houses. The formula can be write like `rob(i) = rob(i-2) + v[i]`.
  * Don't steal the house i, in which case you're free to maximum the stolen value up to house `i-1`. And you add nothing to you wealth. The formula can be write like `rob(i) = rob(i - 1)`.

2. The second step, think about our base case. When the houses are empty, the stolen value will be 0, and is the house is only one, we will return its value, because the better choice is to steal it.

3. The third step, Define a recurrence relation.

Define `rob(i)` to be the result of our problem which is maximum value that can be stolen if only stealing from the houses 0 to i.

**Then our relation formula will be like: `rob(i) = max(rob(i-1), rob(i-2) + v[i]) and (rob(0) = 0, rob(1) = v[1])`.** And the `v[i]` is the value of the house i.

In [52]:
def naive_rob(houses):
    if not houses:
        return 0
    if len(houses) == 1:
        return houses[0]
    return max(naive_rob(houses[2:]) + houses[0], naive_rob(houses[1:]))

In [70]:
timeit.timeit('naive_rob([2,7,9,3,1])', number=100, globals=globals())

0.0005113610532134771

**Use the memorization**

From the relation formula, we know that there will exist the overlapping calculation from `rob(i)`. As `rob(i)` need results of `rob(i-1)` and `rob(i-2)`, and `rob(i-1)`  also need result of `rob(i-2)`. So use the memorization skill to improve our solution will quickly come to our mind. 

In [85]:
def memo_rob(houses, memo={}):
    if not houses:
        return 0
    if len(houses) == 1:
        return houses[0]
    if len(houses) in memo:
        # If subproblem in the memo, do not calculate the same subproblem.
        return memo[len(houses)]
    rob_i = max(memo_rob(houses[2:], memo) + houses[0], memo_rob(houses[1:], memo))
    memo[len(houses)] = rob_i
    return rob_i

In [86]:
timeit.timeit('memo_rob([2,7,9,3,1])', number=1000, globals=globals())

0.00041397102177143097

**Use bottom-up approach**

As the relation formual shows ue `rob(i) = max(rob(i-1), rob(i-2) + v[i]) and (rob(0) = 0, rob(1) = v[1])`, we can notice that, the intermidate produced value can be throwed as its no need to be used on the current calculation.

In [87]:
def bottom_up_rob(houses):
    if not houses:
        return 0
    if len(houses) == 1:
        return houses[0]
    rob_0 = 0
    rob_1 = houses[0]
    for i in range(1, len(houses)):
        tmp = rob_1
        rob_1 = max(rob_0 + houses[i], rob_1)
        rob_0 = tmp
    return rob_1

In [88]:
timeit.timeit('bottom_up_rob([2,7,9,3,1])', number=1000, globals=globals())

0.0015831920318305492

Both of Fibonacci and Robber problem is `one-dimensinal` problems, in which we just need to iterate through a linear sequence of subproblems.
# Coin Change

You are given coins of different denominations and a total amount of money amount. Write a function to compute the fewest number of coins that you need to make up that amount. If that amount of money cannot be made up by any combination of the coins, return -1.

**Example 1:**
```py
Input: coins = [1, 2, 5], amount = 11
Output: 3 
Explanation: 11 = 5 + 5 + 1
```
**Example 2:**
```py
Input: coins = [2], amount = 3
Output: -1
```

1. First, define the subproblem, at any given point in the algorithm, we will consider a subset of the denominations `coins`, and the amount we want to add up to. Let's bring us two choice to make.
  * We not to use the first coin on our coins set. And we use the same target amount to find the lowest number of coins to make the change. Our coins set will extract the first one out. The relation formula will be write as `change(coins, amount) = change(coins[1:], amount)`.
  * Use the first coin and decreases the target amount correspond on the denomination of the first coin. And do not change our coins set, which give us a chance to use the same coin on the next recursion. If we choose this coin to decrease our amount we need to increase the count of change coin by one. The relation formula will be write as `change(coins, amount) = change(coins, amount - coins[0])`.
  
2. Second, figure out our base case where we need to stop our recursive. 
  * When the coins set is one, which means we can only use one coin to change our amount, so if the reminder of amount divide the denomination of coin is zero, the divide value is our count, otherwise, we will return MAX, there is no way to change the amount with this coin.
  * When the amount is zero, there are no need to change our amount anymore, so our count return zero.
  * As we don't know whether our coins set if sorted or not, we need filter the coins set by comparing the first coin denomination with our target amount or not. If it's greater than our target amount, we can eliminate it from our coins set.

3. Third, the recursion relation formula of this problem can be write as `change(coins, amount) = min(change(coins[1:], amount), change(coins, amount - coins[0]))`.

In [149]:
def naive_make_change(coins, amount):
    
    if len(coins) == 1:
        if amount % coins[0]:
            return float('inf')
        count = amount // coins[0]
        return count
    
    if amount == 0:
        return 0
    
    if amount < coins[0]:
        return naive_make_change(coins[1:], amount)
    
    return min(naive_make_change(coins[1:], amount), naive_make_change(coins, amount-coins[0]) + 1)

In [169]:
naive_make_change([1, 2, 5], 11)

3

In [170]:
timeit.timeit('naive_make_change([1, 2, 5], 11)', number=1000, globals=globals())

0.042261141003109515

**Use the memorization**

We use the map to store our recursion result, as its `O(1)` runtime to search our previour calculate value. And we use the current state of our recursion function arguments as the key to store correspond counts.

In [271]:
def memo_make_change(coins, amount, memo={}):
    
    if len(coins) == 1:
        if amount % coins[0]:
            return float('inf')
        count = amount // coins[0]
        return count
    
    if amount == 0:
        return 0
    
    if (coins[0], amount) in memo:
        # If subproblem in the memo, do not calculate the same subproblem.
        return memo[(coins[0], amount)]
    
    if amount < coins[0]:
        return memo_make_change(coins[1:], amount, memo)
    
    change_n = min(memo_make_change(coins[1:], amount, memo), memo_make_change(coins, amount-coins[0], memo) + 1)
    memo[(coins[0], amount)] = change_n
    return change_n

In [272]:
memo_make_change([1, 2, 5], 11)

3

In [273]:
timeit.timeit('memo_make_change([370,417,408,156,143,434,168,83,177,280,117], 9953)', number=1000, globals=globals())

0.11349478806369007

**Bottom up approah**

`F(S)` - minimum number of coins needed to make change for amount S using coin denominations `coins[c0, c1, ... ,cn-1]`.
Our recursion relation can be redefine like, `F(S) = min F(S - ci) + 1 for ci in range coins and S - ci >= 0`. Also our base case can be like `F(S) == 0, when S == 0`. We consider our problem from bottom, when our amount = 1, and our coins set is [1, 2, 3]. the result of `F(1) = F(1 - 1) + 1` will be `1`, and when amount is 2, we will have `F(2) = min (F(2-1), F(2-2))` will is 1.
Ans we can fill out our two dimension table of as following.

|amount\coins |1 |2 |3| F(i)|
|------|:------|:--------|:--------|:---|
|1|F(0)|0|0|1|
|2|F(1)|F(0)|0|1|
|3|F(2)|F(1)|F(0)|1|
|4|F(3)|F(2)|F(1)|2|
|5|F(4)|F(3)|F(2)|2|
|6|F(5)|F(4)|F(3)|2|

In [282]:
def bottom_up_make_change(coins, amount):
    max_count = amount + 1
    dp = [max_count for _ in range(amount+1)]
    dp[0] = 0
    for i in range(1, amount+1):
        for j in range(len(coins)):
            if coins[j] <= i:
                dp[i] = min(dp[i], dp[i - coins[j]] + 1)
    return dp[amount]

In [283]:
bottom_up_make_change([1, 2, 5], 11)

3

In [279]:
timeit.timeit('bottom_up_make_change([370,417,408,156,143,434,168,83,177,280,117], 9953)', number=10, globals=globals())

0.44001925201155245